In [67]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import matplotlib.colors as mcolors

# Set the correct file paths
base_path = r"C:\Users\Wolfrank\Desktop\GiGabyte\CodeWolf\UKDataProject\Data\Combined_Data"
output_path = r"C:\Users\Wolfrank\Desktop\GiGabyte\CodeWolf\UKDataProject"

# Create output directory if it doesn't exist
Path(output_path).mkdir(parents=True, exist_ok=True)

# Read the datasets with selected columns
accidents_df = pd.read_csv(f"{base_path}/combined_accidents.csv",
                          usecols=['Accident_Index', 'Date', 'Time', 'Day_of_Week', 
                                 'Accident_Severity', 'Weather_Conditions', 'Light_Conditions',
                                 'Road_Type', 'Speed_limit', 'Urban_or_Rural_Area',
                                 'Number_of_Vehicles', 'Number_of_Casualties',
                                 'Location_Easting_OSGR', 'Location_Northing_OSGR',
                                 'Longitude', 'Latitude'])

casualties_df = pd.read_csv(f"{base_path}/combined_casualties.csv",
                          usecols=['Accident_Index', 'Age_of_Casualty', 
                                 'Casualty_Severity', 'Casualty_Type',
                                 'Sex_of_Casualty', 'Casualty_Class'])

vehicles_df = pd.read_csv(f"{base_path}/combined_vehicles.csv",
                         usecols=['Accident_Index', 'Vehicle_Type', 
                                'Age_of_Vehicle', 'Age_of_Driver',
                                'Journey_Purpose_of_Driver'])

# Safe time parsing function
def parse_time(time_str):
    try:
        if pd.isna(time_str) or time_str == 'Unknown':
            return np.nan
        return pd.to_datetime(time_str, format='%H:%M').hour
    except:
        return np.nan

# Data preprocessing
accidents_df['Date'] = pd.to_datetime(accidents_df['Date'], format='%d/%m/%Y')
accidents_df['Month'] = accidents_df['Date'].dt.month
accidents_df['Year'] = accidents_df['Date'].dt.year
accidents_df['Hour'] = accidents_df['Time'].apply(parse_time)

# Color palette and style settings
blue_palette = ['#08519c', '#3182bd', '#6baed6', '#bdd7e7', '#eff3ff']
blue_grey_palette = ['#1f77b4', '#7393B3', '#B0C4DE', '#E6E6FA']

# Set the style for all visualizations
plt.style.use('seaborn-darkgrid')
sns.set_palette(blue_palette)
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['axes.facecolor'] = '#f8f9fa'
plt.rcParams['grid.color'] = '#e9ecef'

# Calculate summary statistics
def calculate_statistics():
    return {
        'total_accidents': len(accidents_df),
        'total_casualties': len(casualties_df),
        'fatal_accidents': len(accidents_df[accidents_df['Accident_Severity'] == 1]),
        'serious_accidents': len(accidents_df[accidents_df['Accident_Severity'] == 2]),
        'slight_accidents': len(accidents_df[accidents_df['Accident_Severity'] == 3])
    }
# Define enhanced color palettes
blue_grey_colors = ['#1f77b4', '#2c3e50', '#7393B3', '#95a5a6', '#B0C4DE']
pie_blues = ['#08519c', '#2171b5', '#4292c6', '#6baed6', '#9ecae1']
heatmap_colors = sns.light_palette("#1f77b4", as_cmap=True)

def create_visualizations():
    # 1. Severity by Road Type Heatmap
    plt.figure(figsize=(15, 8))
    road_severity = pd.crosstab(accidents_df['Road_Type'], accidents_df['Accident_Severity'])
    sns.heatmap(road_severity, cmap=heatmap_colors, annot=True, fmt='d')
    plt.title('Accident Severity by Road Type', pad=20, fontsize=14)
    plt.xlabel('Severity', fontsize=12)
    plt.ylabel('Road Type', fontsize=12)
    plt.tight_layout()
    plt.savefig(f'{output_path}/road_severity_heatmap.png', dpi=300, bbox_inches='tight')
    plt.close()

    # 2. Monthly Trends Line Plot
    plt.figure(figsize=(12, 6))
    monthly_trend = accidents_df.groupby(['Month', 'Accident_Severity']).size().unstack()
    monthly_trend.plot(kind='line', marker='o', color=blue_grey_colors[:3])
    plt.title('Monthly Accident Trends by Severity', pad=20, fontsize=14)
    plt.xlabel('Month', fontsize=12)
    plt.ylabel('Number of Accidents', fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.legend(title='Severity')
    plt.tight_layout()
    plt.savefig(f'{output_path}/monthly_trends.png', dpi=300, bbox_inches='tight')
    plt.close()

    # 3. Age Distribution Stacked Bar Chart
    plt.figure(figsize=(15, 8))
    casualties_df['Age_Group'] = pd.cut(casualties_df['Age_of_Casualty'],
                                      bins=[0, 16, 25, 35, 45, 55, 65, 75, 100],
                                      labels=['0-16', '17-25', '26-35', '36-45', '46-55',
                                            '56-65', '66-75', '75+'])
    accident_ages = pd.crosstab(casualties_df['Age_Group'],
                               casualties_df['Casualty_Severity'])
    accident_ages.plot(kind='bar', stacked=True, color=blue_grey_colors[:3])
    plt.title('Casualty Distribution by Age Group and Severity', pad=20, fontsize=14)
    plt.xlabel('Age Group', fontsize=12)
    plt.ylabel('Number of Casualties', fontsize=12)
    plt.legend(title='Severity')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(f'{output_path}/age_distribution.png', dpi=300, bbox_inches='tight')
    plt.close()

    # 4. New Pie Chart for Severity Distribution
    plt.figure(figsize=(10, 10))
    severity_counts = accidents_df['Accident_Severity'].value_counts()
    plt.pie(severity_counts, labels=['Slight', 'Serious', 'Fatal'], 
            colors=pie_blues[:3], autopct='%1.1f%%',
            startangle=90, shadow=True)
    plt.title('Distribution of Accident Severity', pad=20, fontsize=14)
    plt.axis('equal')
    plt.savefig(f'{output_path}/severity_pie.png', dpi=300, bbox_inches='tight')
    plt.close()

    # 5. Time of Day Heatmap
    plt.figure(figsize=(15, 8))
    time_severity = pd.crosstab(accidents_df['Hour'], accidents_df['Accident_Severity'])
    sns.heatmap(time_severity, cmap=heatmap_colors, annot=True, fmt='d')
    plt.title('Accident Severity by Hour of Day', pad=20, fontsize=14)
    plt.xlabel('Severity', fontsize=12)
    plt.ylabel('Hour of Day', fontsize=12)
    plt.tight_layout()
    plt.savefig(f'{output_path}/time_severity.png', dpi=300, bbox_inches='tight')
    plt.close()

    # 6. Weather Impact Box Plot
    plt.figure(figsize=(12, 6))
    sns.boxplot(data=accidents_df, x='Weather_Conditions', y='Number_of_Casualties',
                hue='Accident_Severity', palette=blue_grey_colors[:3])
    plt.title('Casualties by Weather Conditions and Severity', pad=20, fontsize=14)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(f'{output_path}/weather_casualties.png', dpi=300, bbox_inches='tight')
    plt.close()

    # 7. Vehicle Types Bar Chart
    plt.figure(figsize=(12, 6))
    vehicle_counts = vehicles_df['Vehicle_Type'].value_counts().head(10)
    sns.barplot(x=vehicle_counts.index, y=vehicle_counts.values, 
                palette=sns.light_palette('#1f77b4', n_colors=10))
    plt.title('Top 10 Most Common Vehicle Types in Accidents', pad=20, fontsize=14)
    plt.xlabel('Vehicle Type', fontsize=12)
    plt.ylabel('Count', fontsize=12)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(f'{output_path}/vehicle_types.png', dpi=300, bbox_inches='tight')
    plt.close()

    # 8. Urban vs Rural Stacked Bar
    plt.figure(figsize=(10, 6))
    urban_rural = pd.crosstab(accidents_df['Urban_or_Rural_Area'],
                             accidents_df['Accident_Severity'],
                             normalize='index') * 100
    urban_rural.plot(kind='bar', stacked=True, color=blue_grey_colors[:3])
    plt.title('Accident Severity by Area Type (%)', pad=20, fontsize=14)
    plt.xlabel('Area Type (1: Urban, 2: Rural)', fontsize=12)
    plt.ylabel('Percentage', fontsize=12)
    plt.tight_layout()
    plt.savefig(f'{output_path}/urban_rural.png', dpi=300, bbox_inches='tight')
    plt.close()

# Create all visualizations
create_visualizations()
summary_stats = calculate_statistics()

C:\Users\Wolfrank\AppData\Local\Temp\ipykernel_11596\3634310257.py:54: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-darkgrid')


<Figure size 1200x600 with 0 Axes>

<Figure size 1500x800 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

In [74]:

# Generate HTML report
html_content = f"""
<!DOCTYPE html>
<html>
<head>
    <title>UK Traffic Accident Analysis (2015-2018)</title>
    <style>
        body {{ 
            font-family: 'Segoe UI', Arial, sans-serif;
            line-height: 1.6;
            color: #333;
            margin: 0;
            padding-top: 60px;
            background-color: #f8f9fa;
        }}
    .nav-bar {{
    position: fixed;
    top: 0;
    left: 0;
    width: 100%;
    background-color: #000;
    padding: 10px 0;
    z-index: 1000;
    box-shadow: 0 2px 4px rgba(0,0,0,0.1);
}}
        
        .content {{
            max-width: 1200px;
            margin: 0 auto;
            padding: 20px;
        }}
        
        .nav-links {{
            max-width: 1200px;
            margin: 0 auto;
            padding: 0 20px;
        }}
        
        .nav-link {{
            color: white;
            text-decoration: none;
            padding: 10px 20px;
            margin: 0 5px;
            transition: background-color 0.3s ease;
        }}
        
        .nav-link:hover {{
            background-color: #333;
            border-radius: 4px;
        }}
        
        .section {{
            background: white;
            padding: 30px;
            margin: 30px 0;
            border-radius: 8px;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
        }}
        
        h1, h2 {{
            color: #1f77b4;
            margin-bottom: 20px;
        }}
        
        h3 {{
            color: #2c3e50;
            margin: 20px 0 10px 0;
        }}
        
        img {{
            max-width: 100%;
            height: auto;
            margin: 20px 0;
            border-radius: 8px;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
        }}
        
        .intro {{
            background: linear-gradient(135deg, #1f77b4, #2c3e50);
            color: white;
            padding: 40px;
            border-radius: 8px;
            margin-bottom: 30px;
        }}
        
        .intro h1 {{
            color: white;
            margin-bottom: 20px;
        }}
        
        .findings {{
            background-color: #f8f9fa;
            padding: 20px;
            margin: 20px 0;
            border-left: 4px solid #1f77b4;
            border-radius: 4px;
        }}
        
        .stats-container {{
            display: flex;
            flex-wrap: wrap;
            gap: 20px;
            margin: 20px 0;
        }}
        
        .stat-card {{
            background: white;
            padding: 20px;
            border-radius: 8px;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
            flex: 1;
            min-width: 200px;
            text-align: center;
        }}
        
        .stat-value {{
            font-size: 24px;
            font-weight: bold;
            color: #1f77b4;
            margin-bottom: 10px;
        }}
        
        ul, ol {{
            margin: 10px 0;
            padding-left: 20px;
        }}
        
        li {{
            margin-bottom: 8px;
        }}
          .dictionary-section {{
        margin: 30px 0;
    }}

    .table-wrapper {{
        overflow-x: auto;
        margin: 20px 0;
    }}

    .dictionary-table {{
        width: 100%;
        border-collapse: collapse;
        margin: 20px 0;
        background: white;
        box-shadow: 0 1px 3px rgba(0,0,0,0.1);
    }}

    .dictionary-table th,
    .dictionary-table td {{
        padding: 12px 15px;
        text-align: left;
        border-bottom: 1px solid #e0e0e0;
    }}

    .dictionary-table th {{
        background-color: #1f77b4;
        color: white;
        font-weight: 500;
    }}

    .dictionary-table tr:nth-child(even) {{
        background-color: #f8f9fa;
    }}

    .dictionary-table tr:hover {{
        background-color: #f5f5f5;
    }}
    
        .grid-container {{
        display: grid;
        grid-template-columns: repeat(4, 1fr);
        gap: 20px;
        max-width: 1600px;
        margin: 0 auto;
    }}
    .grid-container {{
    display: grid;
    grid-template-columns: repeat(auto-fit, minmax(300px, 1fr));
    gap: 20px;
    max-width: 1600px;
    margin: 0 auto;
    padding: 20px;
}}
.grid-item {{
    position: relative;
    background: white;
    padding: 10px;
    border-radius: 8px;
    box-shadow: 0 2px 5px rgba(0,0,0,0.1);
    overflow: hidden;
}}
.grid-item:hover {{
    z-index: 100;
    box-shadow: 0 8px 20px rgba(0,0,0,0.2);
}}
.image-container {{
    position: relative;
    overflow: hidden;
    border-radius: 4px;
    aspect-ratio: 4/3;
}}
.grid-item img {{
    width: 100%;
    height: 100%;
    object-fit: cover;
    display: block;
    transition: transform 0.3s ease;
}}
.grid-item:hover img {{
    transform: scale(1.5);
}}
.title {{
    font-size: 14px;
    margin-top: 8px;
    text-align: center;
    color: #333;
    padding: 5px;
    background: white;
    position: relative;
    z-index: 2;
}}
  .tableau-container {{
        width: 100%;
        height: 100vh;  /* Full viewport height */
        margin: 20px 0;
        position: relative;
    }}
    
    .tableau-container .tableauPlaceholder {{
        width: 100%;
        height: 100%;
        margin: 0;
        padding: 0;
    }}
        
        .etl-grid-container {{ 
        display: grid; 
        grid-template-columns: repeat(4, 1fr); 
        gap: 20px; 
        margin-top: 20px; 
    }}

    .etl-grid-item {{ 
        display: flex; 
        flex-direction: column; 
        align-items: center; 
    }}

    .etl-grid-image {{ 
        width: 100%; 
        height: auto; 
        border-radius: 8px; 
        box-shadow: 0 2px 4px rgba(0,0,0,0.1); 
        transition: transform 0.3s ease; 
    }}

    .etl-grid-image:hover {{ 
        transform: scale(1.05); 
    }}

    .etl-image-caption {{ 
        text-align: center; 
        margin-top: 10px; 
        font-size: 0.9em; 
        color: #666; 
    }}

    .technical-notes-list {{ 
        margin-bottom: 20px; 
    }}

    .technical-notes-list ul {{ 
        list-style-type: disc; 
        padding-left: 20px; 
    }}

    .technical-notes-list li {{ 
        margin-bottom: 8px; 
    }} 
        .banner-image {{
    margin-top: 100px;
    width: 100%;
    overflow: hidden;
}}

.banner-image img {{
    max-width: 100%;
    height: auto;
    display: block;
    margin: 0 auto;
    box-shadow: 0 2px 4px rgba(0,0,0,0.1);
}}
    </style>
</head>
<body>
  <nav class="nav-bar">
     <div class="nav-links" style="text-align: center; margin-bottom: 15px;">
        <h1 style="color: white; margin: 0 0 10px 0; font-size: 1.5em;">UK Traffic Accident Analysis (2015-2018)</h1>
    </div>
    <div class="nav-links">
        <a href="#introduction" class="nav-link">Introduction</a>
        <a href="#etl-process" class="nav-link">ETL Process</a>
        <a href="#data-documentation" class="nav-link">Data Documentation</a>
        <a href="#findings" class="nav-link"> Analysis</a>
        <a href="#visualizations" class="nav-link"> Visualizations</a>
        <a href="#Tableau" class="nav-link"> Tableau</a>
        <a href="#conclusion" class="nav-link">Conclusions</a>
    </div>
    <div class="nav-links" style="border-top: 1px solid rgba(255,255,255,0.1); margin-top: 10px; padding-top: 10px;">
        <span style="color: white; font-size: 0.9em;">
            Project by Wolfrank Guzman 
            <a href="https://github.com/guzmanwolfrank" class="nav-link" style="display: inline-block; padding: 5px 10px;">github:guzmanwolfrank</a>
            <a href="https://wolfrankguzman.com" class="nav-link" style="display: inline-block; padding: 5px 10px;">website:wolfrankguzman.com</a>
        </span>
    </div>
</nav>
<!-- Banner Image -->
<div class="banner-image" style="margin-top: 100px; width: 100%; text-align: center;">
    <img src="/Data/Output/ukstudy.png" 
         alt="UK Traffic Study Banner" 
         style="max-width: 100%; height: auto; display: block; margin: 0 auto;">
</div>
    <div class="content">
    <div id="introduction" class="intro">
    <h1>UK Traffic Accident Analysis (2015-2018)</h1>
    <p>Road safety remains one of the most critical public health challenges in the United Kingdom, affecting thousands of lives annually through accidents, injuries, and fatalities. This comprehensive analysis examines traffic accident data from 2015 to 2018, seeking to understand the complex interplay of factors that contribute to road incidents and their severity. By analyzing over {summary_stats['total_accidents']:,} accidents, this study provides crucial insights into patterns, risk factors, and potential intervention points for improving road safety.</p>
    
    <p>The significance of this analysis extends beyond mere statistics, touching upon fundamental aspects of public safety, urban planning, and social responsibility. Through detailed examination of temporal patterns, environmental conditions, and demographic factors, we uncover the multifaceted nature of traffic accidents. The findings reveal how various elements - from weather conditions and road types to driver demographics and vehicle characteristics - interact to influence both the likelihood and severity of accidents. This understanding is crucial for developing targeted interventions and evidence-based policies to enhance road safety.</p>
    
    <p>This study employs advanced statistical analysis and visualization techniques to decode complex patterns within the data, focusing particularly on identifying high-risk scenarios and vulnerable populations. By examining the relationship between factors such as time of day, weather conditions, road types, and accident severity, we aim to provide actionable insights for policymakers, urban planners, and road safety professionals. The analysis pays special attention to age-related patterns, environmental impacts, and vehicular factors, offering a comprehensive view of road safety challenges in the UK.</p>
            
    <h3>Research Objectives:</h3>
    <ul>
        <li>Identify temporal and seasonal patterns in accident occurrence</li>
        <li>Analyze the impact of environmental and road conditions</li>
        <li>Evaluate demographic patterns in casualties</li>
        <li>Assess vehicle-specific risk factors</li>
        <li>Develop evidence-based safety recommendations</li>
    </ul>
</div>
<!--------------->
<div id="etl-process" class="section">
    <h2>ETL Process</h2>
    
    <p>A comprehensive ETL process was performed on road safety data spanning multiple years, focusing on three main datasets: Accidents, Casualties, and Vehicles. The process included data validation, standardization, and consolidation, with particular attention to date/time format standardization from UK to US formats.</p>
    
    <h3>Detailed ETL Process</h3>
    
    <div class="findings">
        <h4>1. Data Validation and Structure Assessment</h4>
        <ul>
            <li>Performed initial data comparison between redundant 2015 datasets to ensure data integrity</li>
            <li>Validated matching data between Data\RoadSafetyData_2015 and organized folder structure</li>
            <li>Confirmed 100% match for Accidents, Casualties, and Vehicles datasets</li>
            <li>Decision made to remove redundant 2015 folder after validation</li>
        </ul>
    </div>
    
    <div class="findings">
        <h4>2. File Structure Standardization</h4>
        <p>Implemented consistent naming convention across all datasets:</p>
        <ul>
            <li>Renamed 2017 accident files from "ACC" to "Accidents_2017"</li>
            <li>Standardized Casualties files naming convention across years</li>
            <li>Updated Vehicles files from "Veh.csv" to standardized naming format</li>
            <li>Organized files into appropriate report type directories</li>
        </ul>
    </div>
    
    <div class="findings">
        <h4>3. Data Transformation</h4>
        <p><strong>Date/Time Format Standardization</strong></p>
        <ul>
            <li>Converted all timestamps from UK to US format</li>
            <li>Resolved secondary time format issues related to hours and minutes</li>
            <li>Implemented additional validation checks for datetime consistency</li>
        </ul>
    </div>
    
    <div class="findings">
        <h4>4. Data Cleaning</h4>
        <p>Executed custom data cleaning script to handle:</p>
        <ul>
            <li>Data standardization</li>
            <li>Error correction</li>
            <li>Format consistency</li>
            <li>Generated comprehensive data cleaning report using custom Python code</li>
        </ul>
    </div>
    
    <div class="findings">
        <h4>5. Data Consolidation</h4>
        <p>Combined cleaned datasets into three main categories:</p>
        <ul>
            <li>Accidents master file</li>
            <li>Casualties master file</li>
            <li>Vehicles master file</li>
        </ul>
    </div>
    
   <div id="etl-process" class="section">
    <div class="findings">
        <h4>Technical Notes</h4>
        <div class="technical-notes-list">
            <p><strong>Primary challenge areas addressed:</strong></p>
            <ul>
                <li>Date/time format standardization</li>
                <li>File organization and naming conventions</li>
                <li>Data validation and verification</li>
            </ul>
            
            <p><strong>Custom Python scripts were developed for:</strong></p>
            <ul>
                <li>Data comparison and validation</li>
                <li>Cleaning operations</li>
                <li>Report generation</li>
                <li>Data consolidation</li>
            </ul>
        </div>

        <div class="etl-grid-container">
            <div class="etl-grid-item">
                <img class="etl-grid-image" 
                     src="/Notes/ETLReports/2015accidentscomparisonreport.png" 
                     alt="2015 Accidents Comparison Report">
                <p class="etl-image-caption">Accidents Comparison Report</p>
            </div>
            
            <div class="etl-grid-item">
                <img class="etl-grid-image" 
                     src="/Notes/ETLReports/2015Casualtiescomparisonreport.png" 
                     alt="2015 Casualties Comparison Report">
                <p class="etl-image-caption">Casualties Comparison Report</p>
            </div>
            
            <div class="etl-grid-item">
                <img class="etl-grid-image" 
                     src="/Notes/ETLReports/2015Vehiclescomparisonreport.png" 
                     alt="2015 Vehicles Comparison Report">
                <p class="etl-image-caption">Vehicles Comparison Report</p>
            </div>
            
            <div class="etl-grid-item">
                <img class="etl-grid-image" 
                     src="/Notes/ETLReports/CleaningReport.png" 
                     alt="Python Folder Reader and CSV Cleaning Program">
                <p class="etl-image-caption">Python Folder Reader and CSV Cleaning Program</p>
            </div>
        </div>
    </div>
</div>
    
    <div class="findings">
        <h4>Process Timeline</h4>
        <div class="stats-container">
            <div class="stat-card">
                <div>12:52</div>
                <div>Initial date format standardization</div>
            </div>
            <div class="stat-card">
                <div>14:11</div>
                <div>Data validation of 2015 datasets</div>
            </div>
            <div class="stat-card">
                <div>14:29</div>
                <div>Folder structure optimization</div>
            </div>
            <div class="stat-card">
                <div>14:41</div>
                <div>File renaming and organization</div>
            </div>
            <div class="stat-card">
                <div>15:18</div>
                <div>UK to US time format conversion</div>
            </div>
            <div class="stat-card">
                <div>15:43</div>
                <div>Implementation of data cleaning</div>
            </div>
            <div class="stat-card">
                <div>16:24</div>
                <div>Data consolidation</div>
            </div>
            <div class="stat-card">
                <div>16:49</div>
                <div>Final datetime format resolution</div>
            </div>
        </div>
    </div>
</div>

<!----------------->

<div id="data-documentation" class="section">
  

    <h2>Data Legend</h2>
    
    <div class="dictionary-section">
        <h3>Accident Severity</h3>
        <div class="table-wrapper">
            <table class="dictionary-table">
                <thead>
                    <tr>
                        <th>Code</th>
                        <th>Label</th>
                    </tr>
                </thead>
                <tbody>
                    <tr><td>1</td><td>Fatal</td></tr>
                    <tr><td>2</td><td>Serious</td></tr>
                    <tr><td>3</td><td>Slight</td></tr>
                </tbody>
            </table>
        </div>

        <h3>Weather Conditions</h3>
        <div class="table-wrapper">
            <table class="dictionary-table">
                <thead>
                    <tr>
                        <th>Code</th>
                        <th>Label</th>
                    </tr>
                </thead>
                <tbody>
                    <tr><td>1</td><td>Fine no high winds</td></tr>
                    <tr><td>2</td><td>Raining no high winds</td></tr>
                    <tr><td>3</td><td>Snowing no high winds</td></tr>
                    <tr><td>4</td><td>Fine + high winds</td></tr>
                    <tr><td>5</td><td>Raining + high winds</td></tr>
                    <tr><td>6</td><td>Snowing + high winds</td></tr>
                    <tr><td>7</td><td>Fog or mist</td></tr>
                    <tr><td>8</td><td>Other</td></tr>
                    <tr><td>9</td><td>Unknown</td></tr>
                    <tr><td>-1</td><td>Data missing or out of range</td></tr>
                </tbody>
            </table>
        </div>

        <h3>Road Type</h3>
        <div class="table-wrapper">
            <table class="dictionary-table">
                <thead>
                    <tr>
                        <th>Code</th>
                        <th>Label</th>
                    </tr>
                </thead>
                <tbody>
                    <tr><td>1</td><td>Roundabout</td></tr>
                    <tr><td>2</td><td>One way street</td></tr>
                    <tr><td>3</td><td>Dual carriageway</td></tr>
                    <tr><td>6</td><td>Single carriageway</td></tr>
                    <tr><td>7</td><td>Slip road</td></tr>
                    <tr><td>9</td><td>Unknown</td></tr>
                    <tr><td>12</td><td>One way street/Slip road</td></tr>
                    <tr><td>-1</td><td>Data missing or out of range</td></tr>
                </tbody>
            </table>
        </div>

        <h3>Casualty Type</h3>
        <div class="table-wrapper">
            <table class="dictionary-table">
                <thead>
                    <tr>
                        <th>Code</th>
                        <th>Label</th>
                    </tr>
                </thead>
                <tbody>
                    <tr><td>0</td><td>Pedestrian</td></tr>
                    <tr><td>1</td><td>Cyclist</td></tr>
                    <tr><td>2</td><td>Motorcycle 50cc and under rider or passenger</td></tr>
                    <tr><td>3</td><td>Motorcycle 125cc and under rider or passenger</td></tr>
                    <tr><td>4</td><td>Motorcycle over 125cc and up to 500cc rider or passenger</td></tr>
                    <tr><td>5</td><td>Motorcycle over 500cc rider or passenger</td></tr>
                    <tr><td>8</td><td>Taxi/Private hire car occupant</td></tr>
                    <tr><td>9</td><td>Car occupant</td></tr>
                    <tr><td>10</td><td>Minibus (8 - 16 passenger seats) occupant</td></tr>
                    <tr><td>11</td><td>Bus or coach occupant (17 or more pass seats)</td></tr>
                    <tr><td>16</td><td>Horse rider</td></tr>
                    <tr><td>17</td><td>Agricultural vehicle occupant</td></tr>
                    <tr><td>18</td><td>Tram occupant</td></tr>
                    <tr><td>19</td><td>Van / Goods vehicle (3.5 tonnes mgw or under) occupant</td></tr>
                    <tr><td>20</td><td>Goods vehicle (over 3.5t. and under 7.5t.) occupant</td></tr>
                    <tr><td>21</td><td>Goods vehicle (7.5 tonnes mgw and over) occupant</td></tr>
                    <tr><td>22</td><td>Mobility scooter rider</td></tr>
                    <tr><td>23</td><td>Electric motorcycle rider or passenger</td></tr>
                    <tr><td>90</td><td>Other vehicle occupant</td></tr>
                    <tr><td>97</td><td>Motorcycle - unknown cc rider or passenger</td></tr>
                    <tr><td>98</td><td>Goods vehicle (unknown weight) occupant</td></tr>
                </tbody>
            </table>
        </div>

        <h3>Vehicle Type</h3>
        <div class="table-wrapper">
            <table class="dictionary-table">
                <thead>
                    <tr>
                        <th>Code</th>
                        <th>Label</th>
                    </tr>
                </thead>
                <tbody>
                    <tr><td>1</td><td>Pedal cycle</td></tr>
                    <tr><td>2</td><td>Motorcycle 50cc and under</td></tr>
                    <tr><td>3</td><td>Motorcycle 125cc and under</td></tr>
                    <tr><td>4</td><td>Motorcycle over 125cc and up to 500cc</td></tr>
                    <tr><td>5</td><td>Motorcycle over 500cc</td></tr>
                    <tr><td>8</td><td>Taxi/Private hire car</td></tr>
                    <tr><td>9</td><td>Car</td></tr>
                    <tr><td>10</td><td>Minibus (8 - 16 passenger seats)</td></tr>
                    <tr><td>11</td><td>Bus or coach (17 or more pass seats)</td></tr>
                    <tr><td>16</td><td>Ridden horse</td></tr>
                    <tr><td>17</td><td>Agricultural vehicle</td></tr>
                    <tr><td>18</td><td>Tram</td></tr>
                    <tr><td>19</td><td>Van / Goods 3.5 tonnes mgw or under</td></tr>
                    <tr><td>20</td><td>Goods over 3.5t. and under 7.5t</td></tr>
                    <tr><td>21</td><td>Goods 7.5 tonnes mgw and over</td></tr>
                    <tr><td>22</td><td>Mobility scooter</td></tr>
                    <tr><td>23</td><td>Electric motorcycle</td></tr>
                    <tr><td>90</td><td>Other vehicle</td></tr>
                    <tr><td>97</td><td>Motorcycle - unknown cc</td></tr>
                    <tr><td>98</td><td>Goods vehicle - unknown weight</td></tr>
                    <tr><td>-1</td><td>Data missing or out of range</td></tr>
                </tbody>
            </table>
        </div>
    </div>
</div>



        
        
        
    <h2>Data Dictionary</h2>
    
    <div class="dictionary-section">
        <h3>Accidents Dataset</h3>
        <div class="table-wrapper">
            <table class="dictionary-table">
                <thead>
                    <tr>
                        <th>Field Name</th>
                        <th>Description</th>
                        <th>Data Type</th>
                        <th>Values/Format</th>
                    </tr>
                </thead>
                <tbody>
                    <tr>
                        <td>Accident_Index</td>
                        <td>Unique identifier for each accident</td>
                        <td>String</td>
                        <td>Unique value linking to vehicle and casualty data</td>
                    </tr>
                    <tr>
                        <td>Location_Easting_OSGR</td>
                        <td>Easting location in OSGR format</td>
                        <td>Numeric</td>
                        <td>Grid Reference (-1 for missing data)</td>
                    </tr>
                    <tr>
                        <td>Location_Northing_OSGR</td>
                        <td>Northing location in OSGR format</td>
                        <td>Numeric</td>
                        <td>Grid Reference (-1 for missing data)</td>
                    </tr>
                    <tr>
                        <td>Longitude</td>
                        <td>Longitude in WGS84 format</td>
                        <td>Decimal</td>
                        <td>WGS 1984 coordinate system</td>
                    </tr>
                    <tr>
                        <td>Latitude</td>
                        <td>Latitude in WGS84 format</td>
                        <td>Decimal</td>
                        <td>WGS 1984 coordinate system</td>
                    </tr>
                    <tr>
                        <td>Accident_Severity</td>
                        <td>Severity of the accident</td>
                        <td>Integer</td>
                        <td>1: Fatal, 2: Serious, 3: Slight</td>
                    </tr>
                    <tr>
                        <td>Number_of_Vehicles</td>
                        <td>Number of vehicles involved</td>
                        <td>Integer</td>
                        <td>Count of vehicles</td>
                    </tr>
                    <tr>
                        <td>Number_of_Casualties</td>
                        <td>Number of casualties</td>
                        <td>Integer</td>
                        <td>Count of casualties</td>
                    </tr>
                    <tr>
                        <td>Date</td>
                        <td>Date of accident</td>
                        <td>Date</td>
                        <td>DD/MM/YYYY format</td>
                    </tr>
                    <tr>
                        <td>Time</td>
                        <td>Time of accident</td>
                        <td>Time</td>
                        <td>HH:MM 24-hour format</td>
                    </tr>
                </tbody>
            </table>
        </div>

        <h3>Casualties Dataset</h3>
        <div class="table-wrapper">
            <table class="dictionary-table">
                <thead>
                    <tr>
                        <th>Field Name</th>
                        <th>Description</th>
                        <th>Data Type</th>
                        <th>Values/Format</th>
                    </tr>
                </thead>
                <tbody>
                    <tr>
                        <td>Accident_Index</td>
                        <td>Reference to accident record</td>
                        <td>String</td>
                        <td>Links to accident data</td>
                    </tr>
                    <tr>
                        <td>Vehicle_Reference</td>
                        <td>Reference to vehicle involved</td>
                        <td>Integer</td>
                        <td>Links to vehicle data</td>
                    </tr>
                    <tr>
                        <td>Casualty_Class</td>
                        <td>Type of casualty</td>
                        <td>Integer</td>
                        <td>1: Driver/Rider, 2: Passenger, 3: Pedestrian</td>
                    </tr>
                    <tr>
                        <td>Sex_of_Casualty</td>
                        <td>Gender of casualty</td>
                        <td>Integer</td>
                        <td>1: Male, 2: Female, -1: Unknown</td>
                    </tr>
                    <tr>
                        <td>Age_of_Casualty</td>
                        <td>Age of casualty</td>
                        <td>Integer</td>
                        <td>Age in years (-1 for unknown)</td>
                    </tr>
                </tbody>
            </table>
        </div>

        <h3>Vehicles Dataset</h3>
        <div class="table-wrapper">
            <table class="dictionary-table">
                <thead>
                    <tr>
                        <th>Field Name</th>
                        <th>Description</th>
                        <th>Data Type</th>
                        <th>Values/Format</th>
                    </tr>
                </thead>
                <tbody>
                    <tr>
                        <td>Accident_Index</td>
                        <td>Reference to accident record</td>
                        <td>String</td>
                        <td>Links to accident data</td>
                    </tr>
                    <tr>
                        <td>Vehicle_Type</td>
                        <td>Type of vehicle</td>
                        <td>Integer</td>
                        <td>Various codes for vehicle types</td>
                    </tr>
                    <tr>
                        <td>Age_of_Driver</td>
                        <td>Age of driver</td>
                        <td>Integer</td>
                        <td>Age in years (-1 for unknown)</td>
                    </tr>
                    <tr>
                        <td>Age_of_Vehicle</td>
                        <td>Age of vehicle</td>
                        <td>Integer</td>
                        <td>Age in years (-1 for unknown)</td>
                    </tr>
                </tbody>
            </table>
        </div>
    </div>
</div>
<!----------------->
        <div id="methodology" class="section">
            <h2>Methodology and Data Overview</h2>
            <div class="stats-container">
                <div class="stat-card">
                    <div class="stat-value">{summary_stats['total_accidents']:,}</div>
                    <div>Total Accidents</div>
                </div>
                <div class="stat-card">
                    <div class="stat-value">{summary_stats['fatal_accidents']:,}</div>
                    <div>Fatal Accidents</div>
                </div>
                <div class="stat-card">
                    <div class="stat-value">{summary_stats['serious_accidents']:,}</div>
                    <div>Serious Accidents</div>
                </div>
                <div class="stat-card">
                    <div class="stat-value">{summary_stats['slight_accidents']:,}</div>
                    <div>Slight Accidents</div>
                </div>
            </div>
            <p>This analysis utilizes comprehensive accident data from 2015 to 2018, incorporating multiple data sources to provide a holistic view of road safety incidents.</p>
        </div>
        <div id="data-dictionary" class="section">
<!------------------>
        <div id="findings" class="section">
            <h2>Detailed Analysis and Findings</h2>
            
            <h3>1. Temporal Analysis</h3>
            <img src="time_severity.png" alt="Time of Day Analysis">
            <div class="findings">
                <h4>Key Observations:</h4>
                <ul>
                    <li>Peak accident times coincide with rush hours (7-9 AM and 4-6 PM)</li>
                    <li>Highest severity rates occur during nighttime hours (11 PM - 4 AM)</li>
                    <li>Early morning hours (2-5 AM) show lowest frequency but higher severity</li>
                    <li>Weekend patterns differ significantly from weekday patterns</li>
                </ul>
            </div>

            <h3>2. Weather Impact Analysis</h3>
            <img src="weather_casualties.png" alt="Weather Analysis">
            <div class="findings">
                <h4>Key Findings:</h4>
                <ul>
                    <li>Adverse weather conditions significantly impact accident severity</li>
                    <li>Rain is associated with increased accident frequency but lower average severity</li>
                    <li>Snow and ice show lower frequency but higher severity rates</li>
                    <li>Clear weather accounts for majority of accidents due to higher traffic volume</li>
                </ul>
            </div>

            <h3>3. Road and Speed Analysis</h3>
            <img src="road_severity_heatmap.png" alt="Road Type Analysis">
            <img src="speed_severity.png" alt="Speed Analysis">
            <div class="findings">
                <h4>Critical Insights:</h4>
                <ul>
                    <li>Single carriageways account for highest number of accidents</li>
                    <li>Higher speed limits correlate strongly with increased severity</li>
                    <li>Urban roads show higher frequency but lower severity patterns</li>
                    <li>Motorways show relatively low accident rates despite high speeds</li>
                </ul>
            </div>

            <h3>4. Casualty Analysis</h3>
            <img src="casualty_analysis.png" alt="Casualty Analysis">
            <div class="findings">
                <h4>Demographic Patterns:</h4>
                <ul>
                    <li>Young adults (18-25) show higher representation in accidents</li>
                    <li>Elderly casualties (65+) show higher severity rates</li>
                    <li>Pedestrian casualties show distinct age distribution patterns</li>
                    <li>Cyclist casualties concentrate in urban areas and peak commuting times</li>
                </ul>
            </div>

            <h3>5. Vehicle Type Analysis</h3>
            <img src="vehicle_types.png" alt="Vehicle Analysis">
            <div class="findings">
                <h4>Vehicle-Specific Patterns:</h4>
                <ul>
                    <li>Cars represent the majority of vehicles involved</li>
                    <li>Motorcycles show disproportionately high severity rates</li>
                    <li>Heavy goods vehicles involved in fewer but more severe accidents</li>
                    <li>Public transport shows lower involvement rates</li>
                </ul>
            </div>
        </div>
        <h3>6. Age Distribution Analysis</h3>
<img src="age_distribution.png" alt="Age Distribution Analysis">
<div class="findings">
    <h4>Age-Related Patterns:</h4>
    <ul>
        <li>Young adults (17-25) show highest accident involvement</li>
        <li>Severity rates increase with age groups</li>
        <li>Children under 16 show distinct accident patterns</li>
        <li>Middle-aged groups show more moderate risk levels</li>
    </ul>
    
</div>
<!------>
<div id="visualizations" class="section">
  <div><h1> Accident Visualizations </h1></div>

  <div class="grid-container">
        <div class="grid-item">
            <a href="/Data/Output/hourly_distribution.png" target="_blank">
                <img src="/Data/Output/hourly_distribution.png" alt="Hourly Distribution">
            </a>
            <p>Hourly Distribution</p>
        </div>
        <div class="grid-item">
            <a href="/Data/Output/monthly_severity_trends.png" target="_blank">
                <img src="/Data/Output/monthly_severity_trends.png" alt="Monthly Severity Trends">
            </a>
            <p>Monthly Severity Trends</p>
        </div>
        <div class="grid-item">
            <a href="/Data/Output/severity_heatmap.png" target="_blank">
                <img src="/Data/Output/severity_heatmap.png" alt="Severity Heatmap">
            </a>
            <p>Severity Heatmap</p>
        </div>
        <div class="grid-item">
            <a href="/Data/Output/time_of_day.png" target="_blank">
                <img src="/Data/Output/time_of_day.png" alt="Time of Day">
            </a>
            <p>Time of Day</p>
        </div>
        <div class="grid-item">
            <a href="/Data/Output/time_severity.png" target="_blank">
                <img src="/Data/Output/time_severity.png" alt="Time Severity">
            </a>
            <p>Time Severity</p>
        </div>
        <div class="grid-item">
            <a href="/Data/Output/vehicle_types.png" target="_blank">
                <img src="/Data/Output/vehicle_types.png" alt="Vehicle Types">
            </a>
            <p>Vehicle Types</p>
        </div>
    </div>

<!-Tableau---->
<div id="Tableau" class="section">
            <h2>Tableau: Interactive Visualization</h2>
            
            <!-- First Tableau Dashboard -->
            <div class="tableau-container">
                <div class='tableauPlaceholder' id='viz1734461709351' style='position: relative'>
                    <noscript>
                        <a href='#'>
                            <img alt='Location, Time of Day and Year ' 
                                 src='https://public.tableau.com/static/images/uk/uksafetydata/LocationTimeofDayandYear/1_rss.png' 
                                 style='border: none' />
                        </a>
                    </noscript>
                    <object class='tableauViz' style='display:none;'>
                        <param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' />
                        <param name='embed_code_version' value='3' />
                        <param name='site_root' value='' />
                        <param name='name' value='uksafetydata/LocationTimeofDayandYear' />
                        <param name='tabs' value='no' />
                        <param name='toolbar' value='yes' />
                        <param name='static_image' value='https://public.tableau.com/static/images/uk/uksafetydata/LocationTimeofDayandYear/1.png' />
                        <param name='animate_transition' value='yes' />
                        <param name='display_static_image' value='yes' />
                        <param name='display_spinner' value='yes' />
                        <param name='display_overlay' value='yes' />
                        <param name='display_count' value='yes' />
                        <param name='language' value='en-US' />
                        <param name='showVizHome' value='no' />
                        <param name='device' value='desktop' />
                    </object>
                </div>
                <script type='text/javascript'>
                    var divElement = document.getElementById('viz1734461709351');
                    var vizElement = divElement.getElementsByTagName('object')[0];
                    vizElement.style.width = '100%';
                    vizElement.style.height = '100vh';
                    var scriptElement = document.createElement('script');
                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';
                    vizElement.parentNode.insertBefore(scriptElement, vizElement);
                </script>
            </div>

            <!-- Second Tableau Dashboard -->
           <!-- Second Tableau Dashboard -->
<div class="tableau-container">
    <div class='tableauPlaceholder' id='viz1734479402971' style='position: relative'>
        <noscript>
            <a href='#'>
                <img alt='Dashboard 1' 
                     src='https://public.tableau.com/static/images/uk/uksafetydata2/Dashboard1/1_rss.png' 
                     style='border: none' />
            </a>
        </noscript>
        <object class='tableauViz' style='display:none;'>
            <param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' />
            <param name='embed_code_version' value='3' />
            <param name='site_root' value='' />
            <param name='name' value='uksafetydata2/Dashboard1' />
            <param name='tabs' value='no' />
            <param name='toolbar' value='yes' />
            <param name='static_image' value='https://public.tableau.com/static/images/uk/uksafetydata2/Dashboard1/1.png' />
            <param name='animate_transition' value='yes' />
            <param name='display_static_image' value='yes' />
            <param name='display_spinner' value='yes' />
            <param name='display_overlay' value='yes' />
            <param name='display_count' value='yes' />
            <param name='language' value='en-US' />
            <param name='filter' value='publish=yes' />
        </object>
    </div>
    <script type='text/javascript'>
        var divElement = document.getElementById('viz1734479402971');
        var vizElement = divElement.getElementsByTagName('object')[0];
        vizElement.style.width = '100%';
        vizElement.style.height = '100vh';
        var scriptElement = document.createElement('script');
        scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';
        vizElement.parentNode.insertBefore(scriptElement, vizElement);
    </script>
</div>
<!----->

       <div id="conclusion" class="section">
    <h2>Conclusions and Recommendations</h2>
    
    <div class="findings">
        <h3>Statistical Overview</h3>
        <div class="stats-container">
            <div class="stat-card">
                <div class="stat-value">529,294</div>
                <div>Total Accidents Analyzed</div>
            </div>
            <div class="stat-card">
                <div class="stat-value">6,658</div>
                <div>Fatal Accidents</div>
            </div>
            <div class="stat-card">
                <div class="stat-value">87,462</div>
                <div>Serious Accidents</div>
            </div>
            <div class="stat-card">
                <div class="stat-value">435,174</div>
                <div>Slight Accidents</div>
            </div>
        </div>
    </div>

    <div class="findings">
        <h3>Key Vulnerable Road User Groups</h3>
        <ul>
            <li><strong>Pedestrians</strong>
                <ul>
                    <li>Highest risk during urban rush hours</li>
                    <li>Children and elderly most vulnerable</li>
                    <li>Poor visibility in dark conditions increases risk</li>
                    <li>Crossing points are critical incident locations</li>
                </ul>
            </li>
            <li><strong>Cyclists</strong>
                <ul>
                    <li>Urban intersection conflicts predominate</li>
                    <li>Limited visibility during peak hours</li>
                    <li>Lack of dedicated infrastructure</li>
                    <li>Vehicle-cyclist awareness gaps</li>
                </ul>
            </li>
            <li><strong>Young Drivers (17-25)</strong>
                <ul>
                    <li>Inexperience in complex traffic situations</li>
                    <li>Night driving risks</li>
                    <li>Peer pressure and distraction</li>
                    <li>Speed-related incidents</li>
                </ul>
            </li>
            <li><strong>Elderly Road Users</strong>
                <ul>
                    <li>Higher severity rates in accidents</li>
                    <li>Reduced reaction times</li>
                    <li>Medical conditions affecting driving</li>
                    <li>Difficulty with complex junctions</li>
                </ul>
            </li>
            <li><strong>Motorcyclists</strong>
                <ul>
                    <li>High severity rates in accidents</li>
                    <li>Vulnerability in poor weather</li>
                    <li>Junction collision risks</li>
                    <li>Speed-related incidents</li>
                </ul>
            </li>
        </ul>
    </div>

    <div class="findings">
        <h3>Recommended Solutions</h3>
        <ol>
            <li><strong>Smart Infrastructure Implementation</strong> (15-20% potential reduction in urban accidents)
                <ul>
                    <li>AI-powered traffic management systems</li>
                    <li>Dynamic speed limits based on conditions</li>
                    <li>Connected vehicle infrastructure</li>
                </ul>
            </li>
            <li><strong>Enhanced Education Programs</strong> (10-15% potential reduction)
                <ul>
                    <li>Continuous learning systems</li>
                    <li>Virtual reality hazard training</li>
                    <li>Vulnerable user awareness training</li>
                </ul>
            </li>
            <li><strong>Technology-Based Solutions</strong> (20-25% potential reduction)
                <ul>
                    <li>Advanced driver assistance systems</li>
                    <li>Vehicle-to-vehicle communication</li>
                    <li>Automated emergency braking</li>
                </ul>
            </li>
            <li><strong>Time-Based Interventions</strong> (15-20% potential reduction)
                <ul>
                    <li>Rush hour management</li>
                    <li>School zone programs</li>
                    <li>Night-time safety measures</li>
                </ul>
            </li>
            <li><strong>Road Design Improvements</strong> (25-30% potential reduction)
                <ul>
                    <li>Segregated lanes</li>
                    <li>Junction redesigns</li>
                    <li>Enhanced surface materials</li>
                </ul>
            </li>
        </ol>
    </div>

<!-- Previous sections remain unchanged until after Expected Impact -->

            <div class="findings">
                <h3>Expected Impact</h3>
                <p>Through this comprehensive approach, combining technology, education, and infrastructure improvements, we project a potential reduction in overall accident rates by 30-40% over a five-year period. Success will require substantial government investment, public-private partnerships, and continuous monitoring and adaptation of strategies.</p>
            </div>

            <div class="findings">
                <h3>Conclusion</h3>
                <p>The comprehensive analysis of UK traffic accidents from 2015-2018 reveals a complex interplay between temporal, environmental, and human factors in road safety. The data demonstrates both predictable patterns, such as rush hour accident frequency, and counterintuitive insights, like the lower frequency but higher severity of nighttime accidents. Each vulnerable road user group shows distinct risk patterns requiring targeted interventions, from enhanced pedestrian crossing systems to advanced rider training programs for motorcyclists. The analysis particularly highlights the need for age-specific approaches, with young adults being most frequently involved in accidents while elderly road users face higher severity rates.</p>

                <p>The environmental and infrastructure findings underscore the critical role of road design and weather conditions in accident outcomes. The analysis supports the implementation of smart infrastructure solutions, including AI-powered traffic management systems, dynamic speed limits, and connected vehicle infrastructure. The data suggests that comprehensive solutions combining technology, education, and infrastructure improvements could potentially reduce overall accident rates by 30-40% over a five-year period. Key initiatives should include mandatory advanced driver assistance systems, enhanced driver education programs, and targeted time-based interventions for high-risk periods.</p>

                <p>Looking forward, the study advocates for a multi-faceted approach to road safety improvement through five key areas: smart infrastructure implementation, enhanced driver education, technology-based safety systems, targeted time-based interventions, and comprehensive road design overhaul. Success requires substantial government investment, public-private partnerships, and community engagement, implemented through a phased approach over short-term (0-12 months), medium-term (1-3 years), and long-term (3-5 years) periods. This structured approach, combined with regular effectiveness monitoring and adaptation to new technologies, presents a viable pathway to significantly reducing both the frequency and severity of road accidents across the UK. The findings emphasize that while certain risk factors remain constant, the solutions must evolve with technology and changing traffic patterns to effectively address road safety challenges in the modern era.</p>
            </div>
        </div>
    </div>
</body>
</html>
"""

# Save the report
with open(f'{output_path}/report.html', 'w') as f:
    f.write(html_content)

print(f"\nAnalysis complete! Output files have been saved to: {output_path}")
print("Open 'report.html' in a web browser to view the complete analysis.")


Analysis complete! Output files have been saved to: C:\Users\Wolfrank\Desktop\GiGabyte\CodeWolf\UKDataProject
Open 'report.html' in a web browser to view the complete analysis.
